#### This is a personal project of Yannis Stergiou, based on data analysis, data cleaning, regression analysis and machine learning.

##### 🪄 Starting by loading the .csv file from the current working directory and displaying some information about the data.

In [ ]:
import pandas as pd
import numpy  as np


air_data = pd.read_csv("original_listings.csv")
air_data.info()

In [ ]:
for i in air_data:
    print(f"Information about {i} class;")
    print(air_data[f"{i}"].describe())
    print("")

##### ❗ It's necessary to choose witch data we are going to work with.

In [ ]:
air_data_cl = air_data[["host_response_time",
                        "host_response_rate",
                        "host_is_superhost",
                        "latitude",
                        "longitude",
                        "room_type",
                        "accommodates",
                        "bathrooms",
                        "bedrooms",
                        "price",
                        "review_scores_rating"]].copy()

##### ❗ The price class has items in $XX format so we have to keep only the value

In [ ]:
# Remove "$" and ","
air_data_cl["price"] = air_data_cl["price"].str.replace("$","").str.replace(",","")
print(air_data_cl["price"].describe())

##### 🪄 Μετατρέπουμε την μορφή των δεδομένων σε float από object για να μπορούμε να τα επεξεργαστούμε

In [ ]:
# Convert "object" to "float"
air_data_cl["price"] = air_data_cl["price"].astype(float)
print(air_data_cl["price"].describe())

##### 📐 Βλέπουμε εδώ στατιστικές πληροφορίες σχετικά με την χιλιομετρική απόσταση του κάθε airBNB από το Σύνταγμα

In [ ]:
latitude_dif_syn       = air_data_cl["latitude"]  - 37.9755          
longitude_dif_syn      = air_data_cl["longitude"] - 23.7349
mean_distance_syn      = np.deg2rad(air_data_cl["latitude"] + 37.9755) / 2

dist_lat_syntagma      = latitude_dif_syn  * (np.pi / 180) * 6371
dist_long_syntagma     = np.deg2rad(longitude_dif_syn) * 6371 * np.cos(mean_distance_syn)

distance_from_syntagma = np.sqrt(np.power(dist_lat_syntagma, 2) + np.power(dist_long_syntagma, 2))

print("Στατιστικές πληροφορίες σχετικά με την χιλιομετρική απόσταση κάθε airBNB με το Σύνταγμα:")
print(distance_from_syntagma.describe())

air_data_cl["distance_from_syntagma"] = distance_from_syntagma

##### 👁️ Ρίχνουμε μία ματιά στο νέο data frame που έχουμε κατασκευάσει

In [ ]:
grammes, stiles = air_data_cl.shape
print(f"Το νέο μας πλαίσιο δεδομένων έχει {grammes} γραμμές και {stiles} στήλες ή κατηγορίες")
air_data_cl.info()
air_data_cl.describe()

##### 💾 Save the new data frame into a csv file

In [ ]:
air_data_cl.to_csv("cleaned_listings.csv", index = False)
print("File created and saved succesfully")

### 📔 Statistican Analysis

##### Ξεκινάμε βλέπωντας εάν έχουμε outliers στα δεδομένα μας. Η κατηγορία που θα εξετάσουμε είναι η "price" η οποία έχει και την μεγαλύτερη διασπορά

In [ ]:
import matplotlib.pyplot as plt

working_class              = "price"
price_plot, (plot1, plot2) = plt.subplots(1, 2, figsize = (12,5))

plot1.boxplot(air_data_cl[working_class].dropna())
plot1.set_title(f"BoxPlot of {working_class}")
plot1.set_xlabel("")
plot1.set_ylabel("Price in $")

plot2.hist(air_data_cl[working_class].dropna(), color = "yellow", edgecolor = "black")
plot2.set_xlabel("Price in $")
plot2.set_ylabel("Frequency")
plot2.set_title(f"Histogram of {working_class}")

plt.show()

##### ⌨️ Βρήσκουμε τα ouliers

In [ ]:
unique_price, mean_price, std_price, min_price, price_25_per , price_50_per, price_75_per, max_price = air_data_cl["price"].describe()
outlier_killer_upper = price_75_per + 1.5 * (price_75_per - price_25_per)
outlier_killer_lower = price_25_per - 1.5 * (price_75_per - price_25_per)

price_outliers_01    = air_data_cl["price"] <= outlier_killer_upper
price_outliers_02    = air_data_cl["price"] >= outlier_killer_lower
price_outliers       = price_outliers_01 * price_outliers_02
print(price_outliers)

##### 💣 Τώρα αφαιρούμε τα outliers από το data frame μας και βλέπουμε την επίδραση της διαδικασίας αυτής στα δεδομένα μας

In [ ]:
air_data_cl1      = air_data_cl[price_outliers]
grammes1, stiles1 = air_data_cl1.shape

print(f"Το νέο μας πιο καθαρό data frame έχει {grammes1} γραμμές και {stiles1} στήλες")
print(f"Έχουμε μία μείσωση της τάξης του {((grammes - grammes1) * 100) / grammes:.2f}%")
print(f"Η τυπική απόκληση των τιμων 'price' μέσα στο data frame μας ήταν:        {air_data_cl["price"].std():.2f}")
print(f"Και μετά την αφαίρεση των outliers καταφέραμε να την μειώσουμε στην τιμή: {air_data_cl1["price"].std():.2f}")
air_data_cl1.describe()


##### 💾 Αποθήκευση του νέου data frame σε .csv αρχείο

In [ ]:
air_data_cl1.to_csv("cleaned_listings1.csv", index = False)
print("File created and saved succesfully")

##### 👁️ Θα κάνουμε άλλο ένα boxplot για να δούμε και οπτικά το αποτέλεσμα μετά την αφαίρεση των outliers

In [ ]:
working_class = "price"
price_plot, (plot1, plot2) = plt.subplots(1, 2, figsize = (12,5))

plot1.boxplot(air_data_cl1[working_class].dropna())
plot1.set_title(f"BoxPlot of {working_class}")
plot1.set_xlabel("")
plot1.set_ylabel("Price in $")

plot2.hist(air_data_cl1[working_class].dropna(), color = "yellow", edgecolor = "black", bins = 30)
plot2.set_xlabel("Price in $")
plot2.set_ylabel("Frequency")
plot2.set_title(f"Histogram of {working_class}")

plt.show()

##### 📝 Παρατηρούμε πως έχουμε πιο ξεκάθαρη εικόνα πλέον μέσα στα δεοδομένα που αφορούν τις τιμές